In [2]:
# Dependencies
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

which: no chromedriver in (/c/Users/Wei_l/Anaconda3/envs/pythondata:/c/Users/Wei_l/Anaconda3/envs/pythondata/Library/mingw-w64/bin:/c/Users/Wei_l/Anaconda3/envs/pythondata/Library/usr/bin:/c/Users/Wei_l/Anaconda3/envs/pythondata/Library/bin:/c/Users/Wei_l/Anaconda3/envs/pythondata/Scripts:/c/Users/Wei_l/Anaconda3/envs/pythondata/bin:/c/Users/Wei_l/Anaconda3/Scripts/condabin:/c/Users/Wei_l/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/Wei_l/bin:/c/Windows/system32:/c/Windows:/c/Windows/System32/Wbem:/c/Windows/System32/WindowsPowerShell/v1.0:/c/Windows/System32/OpenSSH:/c/Program Files/PuTTY:/cmd:/c/Users/Wei_l/Anaconda3:/c/Users/Wei_l/Anaconda3/Library/mingw-w64/bin:/c/Users/Wei_l/Anaconda3/Library/usr/bin:/c/Users/Wei_l/Anaconda3/Library/bin:/c/Users/Wei_l/Anaconda3/Scripts:/c/Users/Wei_l/AppData/Local/Microsoft/WindowsApps:/c/Users/Wei_l/AppData/Local/Programs/Microsoft VS Code/bin:/c/Program Files/MongoDB/Server/4.2/bin:/usr/bin/vendor_perl:/usr/bi

# Scrape - NASA Mars News Title and Content

In [4]:

# Visit url for NASA Mars News -- Latest News
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html


In [8]:
soup = bs(html, "html.parser")
# Extract article title and paragraph text
article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)




AttributeError: 'NoneType' object has no attribute 'find'

# Scrape - JPL Mars Space Image

In [9]:
# Visit url for JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

# Go to 'FULL IMAGE'
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)
browser.click_link_by_partial_text('more info')
html = browser.html
image_soup = bs(html, 'html.parser')

# Scrape the URL
feat_img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{feat_img_url}'
print(featured_image_url)


C:\Users\Wei_l\Anaconda3\envs\pythondata\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17978_hires.jpg


# Scrape - Mars Weather

In [10]:
# Visit Twitter url for latest Mars Weather
tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
tweet_container = soup.find_all('div', class_="js-tweet-text-container")

# Loop through latest tweets and find the tweet that has weather information
for tweet in tweet_container: 
    mars_weather = tweet.find('p').text
    if 'sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

# Scrape - Mars Facts

In [13]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

table = pd.read_html(facts_url)
mars_facts = table[1]

mars_facts.columns = ['Description','Mars','Earth']
mars_facts.set_index('Description', inplace=True)
mars_facts

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-153 to 20 °C,-88 to 58°C


# DataFrames as HTML

In [14]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')


# Scrape - Mars Hemispheres

In [15]:

# Visit USGS webpage for Mars hemispehere images
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

soup = bs(html, "html.parser")

hemisphere_image_urls = []

results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})


print(hemisphere_image_urls)

AttributeError: 'NoneType' object has no attribute 'find_all'